In [ ]:
from tabulate import tabulate
from typing import Tuple, Union
import json
import numpy as np

In [ ]:
# Set the path to the folder which contains the metrics files
#path_metrics = "./modelEvaluationBackup/SchistosomaMansoni_12_02_2023__best/logs/"
path_metrics = "../../../DataBackupCode/ModelBackups_16_02_23/EnetEvals/modelEvaluationWithoutDeepRed/logs/"

# Set which epoch should be shown
epoch = 10

In [ ]:
def create_table(settings: dict) -> Tuple[list]:
    setting_keys = list(settings.keys())
    setting_values = list(settings.values())
    setting_keys.insert(0, '\033[1;3m' + 'Epoch' + '\033[0m')
    setting_values.insert(0, '\033[1;3m' + 'Mean' + '\033[0m')

    zips_tuples = list(zip(setting_keys, setting_values))  # Output: list with tuples
    zips_lists = list(map(list, zips_tuples))  # Output: list with lists

    return zips_lists


def epoch_in_means(file: str) -> Tuple[float, float]:
    class0_vals = []
    class1_vals = []
    
    for key in file:
        class0_vals.append(file[key][epoch_in_fold - 1][0])
        class1_vals.append(file[key][epoch_in_fold - 1][1])
        
    class0_mean = np.mean(class0_vals)
    class1_mean = np.mean(class1_vals)
    
    return class0_mean, class1_mean


def get_means_per_epoch(file: str) -> dict:
    range_epochs = [*range(eval_every, epochs_per_fold + 1, eval_every)]
    epoch_number = 0
    mean_scores = load_file(file)
    means_per_epoch = {}

    for epoch in range(len(range_epochs)):
        means = 0
        
        for epoche in mean_scores:
            means += (mean_scores[epoche][epoch][0] + mean_scores[epoche][epoch][1] ) / 2

        means_per_epoch[range_epochs[epoch]] = (means / len(mean_scores) )

    return means_per_epoch
    
    
def load_file(filename: str) -> Union[dict, list]:
    data_file = open(path_metrics + filename)
    data_json = json.load(data_file)
    data_file.close()

    return data_json

In [ ]:
settings = load_file('settings.json')
epochs_per_fold = settings['epochs_per_fold']
eval_every = settings['eval_every']

In [ ]:
number_of_evals = epochs_per_fold / eval_every
epoch_in_fold = int(epoch / number_of_evals)

In [ ]:
means_dicescore = load_file('diceScoreMeans.json')
class0_mean, class1_mean = epoch_in_means(means_dicescore)
print("\033[1;3mDice Score Means\033[0m\nMean values of epoch", epoch, ":\nclass 0: ", class0_mean, "\nclass 1: ", class1_mean)

In [ ]:
means_jaccard = load_file('jaccardIndexMeans.json')
class0_mean, class1_mean = epoch_in_means(means_jaccard)
print("\033[1;3mJaccard Index Means\033[0m\nMean values of epoch", epoch, ":\nclass 0: ", class0_mean, "\nclass 1: ", class1_mean)

In [ ]:
means_per_epoch_jaccard = get_means_per_epoch('jaccardIndexMeans.json')
means_per_epoch_dice = get_means_per_epoch('diceScoreMeans.json')
tabledata = create_table(means_per_epoch_jaccard)
print("JACCARD")
print(tabulate(tabledata, headers='firstrow', tablefmt='fancy_grid', colalign=("left","left")))
tabledata = create_table(means_per_epoch_dice)
print("DICE")
print(tabulate(tabledata, headers='firstrow', tablefmt='fancy_grid', colalign=("left","left")))